# 예측모델 만들기

## Preprocessing

### improt library

In [1]:
import sklearn
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 데이터 가져오기

In [65]:
data = pd.read_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/lotte_drama.csv')
data.rename(columns = {'CLAC1_NM':'분류','CLAC2_NM':'중분류'},inplace = True)
data = pd.get_dummies(data,columns = ['분류','중분류'])
target = pd.read_csv('gdrive/My Drive/공모전/Lpoint/선호지수 만들기/score1.csv')
target2 = pd.read_csv('gdrive/My Drive/공모전/Lpoint/선호지수 만들기/score2.csv')
trainset = pd.merge(data,target, how = 'inner', on=['LABEL']).set_index('LABEL')
trainset2 = pd.merge(data,target2, how = 'inner', on=['LABEL']).set_index('LABEL')
trainset.head()

,PD_BUY_AM,PD_BUY_CT,PD_SALES,BRA_SC,CLAC3_SC,F_10,F_20,F_30,F_40,F_50,...,중분류_여성의류하의,중분류_여성지갑,중분류_여성화,중분류_여행용가방류,중분류_우산/양산류,중분류_유아동화,중분류_캐쥬얼가방,중분류_패션액세서리,중분류_향수,SCORE
LABEL,,,,,,,,,,,,,,,,,,,,,
갤럭시_남성가디건,239200,1,239200,-0.291577,-0.090297,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.055195
게스_남성가디건,63146,144,9093024,0.551194,-0.090297,0.0,3.0,22.0,72.0,22.0,...,0,0,0,0,0,0,0,0,0,0.291875
꼼데가르송_남성가디건,466000,2,932000,-0.291577,-0.090297,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.329767
내셔널지오그래픽_남성가디건,84257,7,589799,-0.291577,-0.090297,0.0,1.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.396146
라코스테_남성가디건,248476,68,16896368,6.421080,-0.090297,0.0,10.0,17.0,13.0,6.0,...,0,0,0,0,0,0,0,0,0,0.408730


In [66]:
data2 = pd.read_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/mlotte_drama.csv')
data2.rename(columns = {'CLAC1_NM':'분류','CLAC2_NM':'중분류'},inplace = True)
data2 = pd.get_dummies(data2,columns = ['분류','중분류'])
target21 = pd.read_csv('gdrive/My Drive/공모전/Lpoint/선호지수 만들기/score12.csv')
target22 = pd.read_csv('gdrive/My Drive/공모전/Lpoint/선호지수 만들기/score22.csv')
testset = pd.merge(data2,target21, how = 'inner', on=['LABEL']).set_index('LABEL')
testset2 = pd.merge(data2,target22, how = 'inner', on=['LABEL']).set_index('LABEL')
testset.head()

,PD_BUY_AM,PD_BUY_CT,PD_SALES,BRA_SC,CLAC3_SC,F_10,F_20,F_30,F_40,F_50,...,중분류_여성의류하의,중분류_여성지갑,중분류_여성화,중분류_여행용가방류,중분류_우산/양산류,중분류_유아동화,중분류_캐쥬얼가방,중분류_패션액세서리,중분류_향수,SCORE
LABEL,,,,,,,,,,,,,,,,,,,,,
2018-04_갤럭시_남성가디건,239200,1,239200,-0.228274,-0.113708,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.052245
2018-04_게스_남성가디건,72507,43,3117801,0.400555,-0.113708,0.0,1.0,5.0,23.0,3.0,...,0,0,0,0,0,0,0,0,0,0.130391
2018-04_내셔널지오그래픽_남성가디건,77400,1,77400,-0.228274,-0.113708,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.095987
2018-04_라코스테_남성가디건,231750,16,3708000,11.259699,-0.113708,0.0,3.0,4.0,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0.276917
2018-04_본_남성가디건,50000,1,50000,1.774020,-0.113708,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.139702


In [67]:
X, y = trainset.iloc[:,:-1], trainset['SCORE']
X.shape, y.shape

((4157, 73), (4157,))

In [68]:
X2, y2 = testset.iloc[:,:-1], testset['SCORE']*100
X2.shape, y2.shape

((16571, 73), (16571,))

## Models

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X2,y2)

### Regression

#### Ridge

In [58]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1, solver='cholesky')
ridge_reg.fit(X_train, y_train)
pred = ridge_reg.predict(X_val)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.137424e-19
  overwrite_a=True).T


In [59]:
from sklearn.metrics import mean_squared_error
print('score = ',ridge_reg.score(X_val,y_val))
print('MSE = ',mean_squared_error(pred,y_val))
print('RMSE = ',mean_squared_error(pred,y_val)**0.5)

score =  0.11667436955006782
MSE =  0.010277087592080637
RMSE =  0.10137597147293158


### Decision Tree

#### Random Forest

In [0]:
from sklearn.ensemble import RandomForestRegressor
rnd_reg = RandomForestRegressor(n_estimators = 100)
rnd_reg.fit(X_train,y_train)
pred = rnd_reg.predict(X_val)

In [73]:
from sklearn.metrics import mean_squared_error
print('score = ',rnd_reg.score(X_val,y_val))
print('MSE = ',mean_squared_error(pred,y_val))
print('RMSE = ',mean_squared_error(pred,y_val)**0.5)

score =  0.5435047704069159
MSE =  53.879792543286264
RMSE =  7.3402855900357356


#### GradientBoost

In [70]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np


gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train,y_train)

errors = [mean_squared_error(y_val,y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=119, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [71]:
from sklearn.metrics import mean_squared_error
pred = gbrt_best.predict(X_val)

print('MSE = ',mean_squared_error(pred,y_val))
print('RMSE = ',mean_squared_error(pred,y_val)**0.5)

MSE =  63.41059979314893
RMSE =  7.963077281626051


In [54]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float('inf')
error_going_up = 0

for n_estimators in range(1,1200):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train,y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val,y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else:
    error_going_up += 1
    if error_going_up == 5:
      print(n_estimators,min_val_error)
      break

350 0.005559259321628956


In [55]:
0.005559259321628956**0.5

0.07456044072850533